In [7]:
#Function 

import statistics

def reorder_decision(daily_sales, lead_time, safety_stock, current_stock):

    reorder_qty = daily_sales * lead_time + safety_stock
    need_reorder = current_stock < reorder_qty

    
    if need_reorder:
        print("need_reorder")

    else:
        print("No reorder needed")

    return reorder_qty, need_reorder

        

reorder_qty, need_reorder = reorder_decision(20,30,170,200)

print(f"Reorderquantity :{reorder_qty}|Decision:{'Yes' if need_reorder else 'No'}")

        

need_reorder
Reorderquantity :770|Decision:Yes


In [15]:
#Class

import statistics

class SKU:
    def __init__(self, name, stock , price):
        self.name = name
        self.stock = stock
        self.price = price

    def sell(self,qty):
        self.stock -= qty
        return qty

    def restock(self,qty):
        self.stock += qty
        return qty

    def stock_value(self):
        stockval = self.stock * self.price
        return stockval



P1 = SKU("Laptop", 120, 55000)




print(f" The product {P1.name} sold {P1.sell(30)} and has restocked {P1.restock(50)} and has stock value of {P1.stock_value()}")

 The product Laptop sold 30 and has restocked 50 and has stock value of 7700000


In [23]:
#Inheritance

class Item:
    def __init__(self, sku, stock , price):
        self.sku = sku
        self.stock = stock
        self.price = price

    def stock_value(self):
        stockval = self.stock * self.price
        return stockval

    def needs_reorder(self, reorder_point):
        need_reor = self.stock < reorder_point

        return need_reor


class PerishableItem(Item):
    def __init__(self, sku, stock, price, expiry):
        super().__init__(sku, stock, price)

        self.expiry = expiry


    def needs_reorder(self,reorder_point):
        if self.expiry <7:
            self.stock < reorder_point *1.5

        else:
            self.stock < reorder_point


P1 = Item("Printer", 120, 15000)
P1.needs_reorder(100)

P2 = PerishableItem("Milk", 120, 60, 3)

P2.needs_reorder(100)
    
print(f"{P1.sku} has {P1.stock} and {'Yes' if P1.needs_reorder else 'No'}")
print(f"{P2.sku} --> {'Yes' if P2.needs_reorder else 'No'}")

Printer has 120 and Yes
Milk --> Yes


In [40]:
#Pandas

import pandas as pd
import numpy as np

data = pd.read_csv("Inventry.csv")


#Analysis
print(data.head())
print(data.isna().count())

data.describe()

pr_avg = data.groupby("Product").mean()
print(pr_avg["ReorderPoint"])



data["stock<reor"] = data.apply(
    lambda row: "Yes" if row["Stock"] < row["ReorderPoint"] else "No", axis=1
)


low_stock = data.loc[data["stock<reor"] == 'No']

print(low_stock)

low_stock.to_csv("lowstock.csv")

      Product  Stock  ReorderPoint
0      Laptop    120           150
1      Tablet     80           100
2      Mobile     50            70
3     Printer     40            60
4  Headphones    200           150
Product         5
Stock           5
ReorderPoint    5
dtype: int64
Product
Headphones    150.0
Laptop        150.0
Mobile         70.0
Printer        60.0
Tablet        100.0
Name: ReorderPoint, dtype: float64
      Product  Stock  ReorderPoint stock<reor
4  Headphones    200           150         No


In [14]:
#Pandas

import pandas as pd
import numpy as np

data = pd.read_csv("ss.csv")
print(data.head())
print(data.isna().count())

data["SafetyStock"] = data["Z"] * data["StdDemand"] * np.sqrt(data["LeadTimeDays"])
data["ReorderPoint"] = data["MeanDemand"] * data["LeadTimeDays"] + data["SafetyStock"]

top = data.sort_values(by = "ReorderPoint", ascending = False)
print(f"Top 3 SKU'S by Reorderpoint \n{top.head(3)}")


print(f"Sum of reorder Point \n {data['ReorderPoint'].sum()}")

print(f"Mean of safety stock :\n{data['SafetyStock'].mean()}")

data.to_csv("reorder_plan.csv")

  SKU  MeanDemand  StdDemand  LeadTimeDays     Z     Category
0   A         100         20            10  1.65  Electronics
1   B          80         10             5  1.28  Electronics
2   C          60         15             7  1.65  Peripherals
3   D         150         30            12  1.96  Electronics
4   E          40         12             4  1.28  Peripherals
SKU             5
MeanDemand      5
StdDemand       5
LeadTimeDays    5
Z               5
Category        5
dtype: int64
Top 3 SKU'S by Reorderpoint 
  SKU  MeanDemand  StdDemand  LeadTimeDays     Z     Category  SafetyStock  \
3   D         150         30            12  1.96  Electronics   203.689175   
0   A         100         20            10  1.65  Electronics   104.355163   
2   C          60         15             7  1.65  Peripherals    65.482345   

   ReorderPoint  
3   2003.689175  
0   1104.355163  
2    485.482345  
Sum of reorder Point 
 4212.868352816502
Mean of safety stock :
86.57367056330047


In [3]:
#Numpy

import numpy as np

weekly = np.array([100,120,110,130,125,115,140,150,135,145,160,155])
lead_time_weeks = 3
safety_stock = 200

std = np.std(weekly)
mean = np.mean(weekly)

cov = std/mean*100
perweekorder = weekly * lead_time_weeks * safety_stock

min_perweek = np.argmin(perweekorder)
max_perweek = np.argmax(perweekorder)
meanperweek = np.mean(perweekorder)
print(perweekorder)

print(min_perweek)
print(max_perweek)
print(meanperweek)


def simulate_stockout_fraction(mean, std, safety_stock, runs=500, weeks=3, days_per_week=7, seed=None):
   
    if seed is not None:
        np.random.seed(seed)

    days = weeks * days_per_week

    # Simulate daily demand for each run (Normal dist)
    daily = np.random.normal(loc=mean, scale=std, size=(runs, days))

    # Optional: clip negatives to zero (if negative demand is not meaningful)
    daily = np.clip(daily, 0, None)

    # Total demand over lead time for each run
    totals = daily.sum(axis=1)

    # Fraction of stockouts (total demand exceeding safety stock)
    fraction = np.mean(totals > safety_stock)

    # 95% CI for the fraction (binomial approximation)
    p = fraction
    n = runs
    se = np.sqrt(p * (1 - p) / n)
    ci_low = max(0.0, p - 1.96 * se)
    ci_high = min(1.0, p + 1.96 * se)

    print(f"Fraction of runs with total > safety_stock: {fraction:.3f}")
    print(f"95% CI: [{ci_low:.3f}, {ci_high:.3f}]")
    return fraction, totals

# ---- Example usage ----
# Change these to your actual numbers:
mean = 20         # average daily demand
std = 5           # std dev of daily demand
safety_stock = 500

simulate_stockout_fraction(mean, std, safety_stock, runs=500, seed=42)


[60000 72000 66000 78000 75000 69000 84000 90000 81000 87000 96000 93000]
0
10
79250.0


In [14]:
#Loop

items = [
  {"sku":"A","days":12},
  {"sku":"B","days":45},
  {"sku":"C","days":110},
]


for item in items:
    

    if item["days"] <=30:
        print(f" The item {item['sku']} --> healthy")
    elif item["days"]>= 31 and item["days"] <=90:
        print(f" The item {item['sku']} --> Slow moving")

    else:
        print(f" the item {item['sku']} --> Dead stock")

    

 The item A --> healthy
 The item B --> Slow moving
 the item C --> Dead stock
